## Explicit triangle

The normal ${\bf n}$ can be calculated in different ways:

1. by back-looking, e.g. at the gradient of $\triangle_{ABD}$, $\tilde{\bf{n}} = \rm{grad} (\triangle_{ABD})$, then normalizing ${\bf n} = {\bf \tilde{n}}/{\| \bf \tilde{n}} \|$
2. by an interpolation or approximation with a broader stencil.
3. local self-reference on the current triangle $\triangle_{ABD}$
4. extended stencil around the current triangle

[We started with option (1.), but now option (3.) seems the more accurate approach, with the perspective of options (2.) and (4.) for achieving higher order].




## Load mesh

In order to test the algorithm with real data we generate a mesh.

In [5]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt
import math

mesh = om.read_trimesh('T4b.off')
V = mesh.points()

X_MAX = 0.5
Y_MIN = 0.6
V[:,0] = V[:,0]*X_MAX / 4
V[:,1] = Y_MIN + (1-Y_MIN)*V[:,1]
X=V[:,0]
Y=V[:,1]
T = X*(1+0.5*(1-Y)**2)
S = X*(1+0.5*(1-Y)**2)

Generate a list of boundary faces, and obtain their indices:

In [7]:
def getBoundaryFaces():
    BoundaryFaceList = []
    for fh in mesh.faces():
        if mesh.is_boundary(fh):
            BoundaryFaceList.append(fh)          
    return BoundaryFaceList

def getIndices(HandleList):
    I_list = []
    for h in HandleList:
        I_list.append(h.idx())        
    return I_list
    
BoundaryFaceList = getBoundaryFaces()
BoundaryFaceIndices = getIndices(BoundaryFaceList)
print(BoundaryFaceIndices)


[0, 2, 8, 9, 18, 19, 22, 28, 29, 30, 31, 33, 37, 39, 40, 53, 54, 55, 61, 68, 69, 70, 74, 76, 80, 85]


In [8]:
def getNodeIndices(fh):
    I = []
    for vh in mesh.fv(fh):
        I.append(vh.idx())            
    return I

fh = BoundaryFaceList[3]
I = getNodeIndices(fh)
print(I)

[32, 52, 3]


Identify the faces at the incoming borders:

In [9]:
print('\t face \t I \t \t X \t \t \t \t Y')
for fh in BoundaryFaceList:
    I = getNodeIndices(fh)
    if 0 in X[I]:
        print('x=0: \t', fh.idx(), ' \t', I, '\t', X[I], '\t', Y[I])
    if 1 in Y[I]:
        print('y=1: \t', fh.idx(), ' \t', I, '\t', X[I], '\t', Y[I])

	 face 	 I 	 	 X 	 	 	 	 Y
x=0: 	 0  	 [15, 10, 38] 	 [0.     0.     0.0625] 	 [0.84000001 0.76       0.8       ]
x=0: 	 2  	 [5, 34, 10] 	 [0.     0.0625 0.    ] 	 [0.68 0.72 0.76]
x=0: 	 18  	 [50, 30, 0] 	 [0.0625 0.0625 0.    ] 	 [0.6  0.64 0.6 ]
x=0: 	 19  	 [30, 5, 0] 	 [0.0625 0.     0.    ] 	 [0.64 0.68 0.6 ]
y=1: 	 39  	 [49, 57, 28] 	 [0.4375 0.4375 0.375 ] 	 [0.95999999 1.         1.        ]
y=1: 	 40  	 [57, 49, 29] 	 [0.4375 0.4375 0.5   ] 	 [1.         0.95999999 1.        ]
x=0: 	 53  	 [46, 54, 25] 	 [0.0625 0.0625 0.    ] 	 [0.95999999 1.         1.        ]
y=1: 	 53  	 [46, 54, 25] 	 [0.0625 0.0625 0.    ] 	 [0.95999999 1.         1.        ]
x=0: 	 54  	 [20, 46, 25] 	 [0.     0.0625 0.    ] 	 [0.92       0.95999999 1.        ]
y=1: 	 54  	 [20, 46, 25] 	 [0.     0.0625 0.    ] 	 [0.92       0.95999999 1.        ]
y=1: 	 55  	 [54, 46, 26] 	 [0.0625 0.0625 0.125 ] 	 [1.         0.95999999 1.        ]
y=1: 	 69  	 [49, 24, 29] 	 [0.4375 0.5    0.5   ] 	 [0.95999999 

We have identified:

* face 2, where $x_A=0, x_B=0, x_C=h_x$
* face 40, where $y_A = 1, y_B= 1, y_C = 1 - h_y$


In [6]:
fh = mesh.face_handle(2)
fh = mesh.face_handle(40)

I = getNodeIndices(fh)
if 0 in X[I] or 1 in Y[I]:
    n = np.array([1, 0])
    print(n)

[1 0]


In a triangle with nodes $A, B, C$, we want to project the value of $C$ from given values in $A, B$.
by the following geometrix construction:
\begin{align}
    {\bf x}(\xi) = (1-\xi) {\bf x}_A + \xi {\bf x}_B \\
    {\bf x}_C = {\bf x}(\xi) + {\bf n} \Delta x ,
\end{align}
which allows to resolve
\begin{align*}
    \frac{\Delta x}{\Delta t} = v
\end{align*}
within the geometry
\begin{align*}
    \Delta t = T_C - T_\xi \\
    \Delta x = \| {\bf x}_C - {\bf x}_\xi \|_2
\end{align*}
If the positions ${\bf x}_A, {\bf x}_B, {\bf x}_C$ and the normal ${\bf n}$ are given,
then the position parameter $\xi$ and the step $\Delta x$ can be calculated by solving the system of equations
\begin{align*}
    \begin{bmatrix}
        {\bf x}_B - {\bf x}_A | {\bf n}
    \end{bmatrix}
    \begin{pmatrix}
        \xi \\
        \Delta x
    \end{pmatrix}
    =
    {\bf x}_C - {\bf x}_A
\end{align*}
With this information various variables can interpolated in the proyection pivot as
\begin{align*}
    y_\xi = (1-\xi) y_A + \xi y_B \\
    S_\xi = (1-\xi) S_A + \xi S_B \\
    T_\xi = (1-\xi) T_A + \xi T_B, 
\end{align*}
such that 
the proyected values in node $C$ are obtained es
\begin{align*}
    S_C = S_\xi + \Delta x \\
    T_C = T_\xi + \Delta x v^{-1}
\end{align*}
The velocity inverse
\begin{align*}
    v^{-1} = w = \frac{S}{y}
\end{align*}
is approximated in the interval $[{\bf x}_\xi, {\bf x}_C]$ by the trapezoide
\begin{align*}
    \frac{1}{\| {\bf x}_C - {\bf x}_\xi \|} \int_{{\bf x}_\xi}^{{\bf x}_C} v^{-1} ({\bf x (\omega)}) \omega 
    %
    \approx \frac{1}{2} \Bigl[ v_\xi^{-1} + v_c^{-1} \Bigr]
    =
    \frac{1}{2} \Bigl[ \frac{S_\xi}{y_\xi} + \frac{S_c}{y_C} \Bigr]
\end{align*}

\begin{align*}
\end{align*}

\begin{align*}
\end{align*}

\begin{align*}
\end{align*}





In [7]:
def getGrad2(I, U):
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = X[I[k+1]] - X[I[0]]
        m[k,1] = Y[I[k+1]] - Y[I[0]]
        b[k] = U[k+1]-U[0]
                
    g = np.linalg.solve(m, b)    
    # c = np.linalg.cond(m)

    return g


def getAngle(v1, v2):
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))    
    return angle

def get_TS_C(I):    
    # swap indices to guarantee order T_A < T_B
    if T[I[0]] > T[I[1]]:
        I[0], I[1] = I[1], I[0]
    
    x_A = V[I[0],0:2]
    x_B = V[I[1],0:2]
    x_C = V[I[2],0:2]
    
    T_A=T[I[0]]
    S_A=S[I[0]]
    T_B=T[I[1]]
    S_B=S[I[1]]
    # T_C=T[I[2]]
    # S_C=S[I[2]]
    
    # 
    x_AB = x_B-x_A
    x_CA = x_C-x_A
    
    #
    U = T[I]
    g = getGrad2(I, U)
    n_A = g / (g[0]**2 + g[1]**2)**0.5
    n_A = np.array([1, 0])
    
    
    #
    M = np.vstack((x_AB,n_A)).transpose()
    c = np.linalg.cond(M)
    if c > 1e9:
        print('ERR: cond', c)
        return c, c, c, c
        # 'inf', 'inf', 'inf', 'inf'
    
    p = np.linalg.solve(M, x_CA)
    xi = p[0]
    delta_X = p[1]    
    
    #
    x_xi = (1-xi)*x_A + xi*x_B
    S_xi = (1-xi)*S_A + xi*S_B
    T_xi = (1-xi)*T_A + xi*T_B
    S_C = S_xi + delta_X
    vel_inv = (S_xi/x_xi[1] + S_C/x_C[1])/2
    T_C = T_xi + delta_X*vel_inv
    
    return xi, delta_X, S_C, T_C

# calculating residual
def res_TS_I(I):
    xi, delta_X, S_C, T_C = get_TS_C(I)
    
    rT = T_C - T[I[2]]
    rS = S_C - S[I[2]] 
    
    return rT, rS
    
global global_I


def residual(U): # res_TS_U_C(U):
    global global_I
    I = global_I
    
    T[I[2]] = U[0]
    S[I[2]] = U[1]

    xi, delta_X, S_C, T_C = get_TS_C(I)    

    rT = T_C - T[I[2]]
    rS = S_C - S[I[2]] 
    
    return rT, rS
    
def cost(u):
    R=residual(u)
    c=np.inner(R,R)
    return c    
    
rT_1, rS_1 = res_TS_I([I[0], I[1], I[2]])



    

In [22]:
def cases(I):
    x_A = V[I[0],0:2]
    x_B = V[I[1],0:2]
    x_C = V[I[2],0:2]
    
    n_A = np.array([1, 0])
    x_AB = x_B-x_A
    x_CA = x_A-x_C
    x_BC = x_C-x_B
    x_AB = x_AB / (x_AB[0]**2 + x_AB[1]**2)**0.5
    x_CA = x_CA / (x_CA[0]**2 + x_CA[1]**2)**0.5
    x_BC = x_BC / (x_BC[0]**2 + x_BC[1]**2)**0.5

    # theta1=getAngle(n_A, x_AB)
    # theta2=getAngle(n_A, x_CA)
    # theta3=getAngle(n_A, x_BC)
    # print(theta1, theta2, theta3)
    
    a1=np.dot(n_A, x_AB)
    a2=np.dot(n_A, x_CA)
    a3=np.dot(n_A, x_BC)
    a = abs(np.array([a1, a2, a3]))
    
    print('ortogonality:', a1, a2, a3)
    argmin_a = np.argmin(a)

    return argmin_a

I = [57, 29, 49] # y=1
I = [5, 10, 34] # x=0
print('I:', I, ', \t X:', X[I], ', \t Y:', Y[I])

cs = cases(I)
print('cases:', cs)

I: [5, 10, 34] , 	 X: [0.     0.     0.0625] , 	 Y: [0.68 0.76 0.72]
ortogonality: 0.0 -0.8422713787794882 0.842271415249527
cases: 0


In [23]:
import scipy.optimize as opt
import time

U = np.zeros(2)
U[0] = T[I[2]]
U[1] = S[I[2]]

U[0] = 1
U[1] = 1

global global_I
global_I = I
r = residual(U)
print(r)

(-0.9972873264068542, -0.9375)


In [24]:
t0 = time.time()
res = opt.minimize(cost, U, method='powell', tol=1e-10, options={'maxiter': 200, 'disp': True})
t1 = time.time()

print('\n t1-t0: = %.1f' % float(t1 - t0),'\n res:', res)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 36

 t1-t0: = 0.0 
 res:    direc: array([[1., 0.],
       [0., 1.]])
     fun: 5.58105909860809e-32
 message: 'Optimization terminated successfully.'
    nfev: 36
     nit: 2
  status: 0
 success: True
       x: array([0.00271267, 0.0625    ])


In [25]:
res.x
res.fun

5.58105909860809e-32

The equation for one triangle can be formulated in an explicit or an implicit way:
\begin{align*}
    \triangle-\text{explicit} \qquad U_C = F_{\rm expl}(U_A, U_B) \\
    \triangle-\text{implicit} \qquad F_{\rm impl}(U_A, U_B; U_C) = 0
\end{align*}
Both formulations are mutually transformable.
The transformation of the implicit equations to the explicit equations can be donde by defining
\begin{align*}
    F_{\rm impl}(U_A, U_B; U_C) := F_{\rm expl}(U_A, U_B) - U_C
\end{align*}
the transformation from the implicit formulation to explicit formulation can be done by employing an optimization algorithm.


In [26]:
I = [57, 49, 29]

xi_1, dX_1, S_C1, T_C1 = get_TS_C(I)

xi_1, dX_1, S_C1, T_C1 = get_TS_C([I[0], I[1], I[2]])
xi_2, dX_2, S_C2, T_C2 = get_TS_C([I[2], I[0], I[1]])
xi_3, dX_3, S_C3, T_C3 = get_TS_C([I[1], I[2], I[0]])


print('xi:', xi_1, ', \t dX:', dX_1, ', \t S_C:', S_C1, ', \t T_C:',T_C1)
print('xi:', xi_2, ', \t dX:', dX_2, ', \t S_C:', S_C2, ', \t T_C:',T_C2)
print('xi:', xi_3, ', \t dX:', dX_3, ', \t S_C:', S_C3, ', \t T_C:',T_C3)
print('--------- \n')

rT_1, rS_1 = residual([I[0], I[1], I[2]])
rT_2, rS_2 = residual([I[2], I[0], I[1]])
rT_3, rS_3 = residual([I[1], I[2], I[0]])

# ', \t rT', rT_1, ', \t rS', rS_1)

if math.isinf(xi_1):
    print('1', xi_1)
    
if math.isinf(xi_2):
    print('2', xi_2)  
    
if math.isinf(xi_3):
    print('3', xi_3)


ERR: cond inf
xi: -0.0 , 	 dX: 0.0625 , 	 S_C: 0.5 , 	 T_C: 0.466796875
xi: inf , 	 dX: inf , 	 S_C: inf , 	 T_C: inf
xi: 1.0 , 	 dX: -0.0625 , 	 S_C: 0.4375 , 	 T_C: 0.470703125
--------- 

2 inf


In [27]:
if True:
    if x_A[0]==0 and x_B[0]==0:
        n_A = np.array([1,0])
    elif x_A[1]==1 and x_B[1]==1:
        n_A = np.array([1,0])
    else:
        n_A = getGrad2(I, U)

    
###################################################

NameError: name 'x_A' is not defined

In [17]:
n_A

NameError: name 'n_A' is not defined